## Manipulating and transforming DataFrames

In [1]:
import warnings

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
from openbb import obb

In [3]:
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

Fetches historical price data for the equity "AAPL" using the "yfinance" provider and stores it in 'asset'

In [4]:
asset = obb.equity.price.historical("AAPL", provider="yfinance")

In [5]:
display(asset)

,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2023-06-20,184.410004,186.100006,184.410004,185.009995,49799100,0.0,0.0
2023-06-21,184.899994,185.410004,182.589996,183.960007,49515700,0.0,0.0
2023-06-22,183.740005,187.050003,183.669998,187.000000,51245300,0.0,0.0
2023-06-23,185.550003,187.559998,185.009995,186.679993,53079300,0.0,0.0
2023-06-26,186.830002,188.050003,185.229996,185.270004,48088700,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-12,207.369995,220.199997,206.899994,213.070007,198134300,0.0,0.0
2024-06-13,214.740005,216.750000,211.600006,214.240005,97862700,0.0,0.0
2024-06-14,213.850006,215.169998,211.300003,212.490005,70122700,0.0,0.0


Fetches historical price data for the equity "SPY" using the "yfinance" provider and stores it in 'benchmark'

In [6]:
benchmark = obb.equity.price.historical("SPY", provider="yfinance")

In [7]:
display(benchmark)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2023-06-20,437.450012,438.369995,435.029999,437.179993,76160400,0.0,0.0,0.0
2023-06-21,436.160004,436.989990,434.329987,434.940002,76982300,0.0,0.0,0.0
2023-06-22,433.950012,436.619995,433.600006,436.510010,70637200,0.0,0.0,0.0
2023-06-23,432.929993,435.059998,432.470001,433.209991,92074500,0.0,0.0,0.0
2023-06-26,432.619995,434.609985,431.190002,431.440002,72823600,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-06-12,541.630005,544.119995,540.299988,541.359985,63251300,0.0,0.0,0.0
2024-06-13,543.150024,543.330017,539.590027,542.450012,44760900,0.0,0.0,0.0
2024-06-14,540.880005,542.809998,539.849976,542.780029,40089900,0.0,0.0,0.0


Updates the column names for 'asset' and 'benchmark'

In [8]:
columns = [
    "open",
    "high",
    "low",
    "close",
    "volume",
    "dividends",
    "splits",
]

In [9]:
asset.columns = columns
benchmark.columns = columns + ["capital_gain"]

Adds a new column 'price_diff' that contains the difference in closing prices

In [10]:
asset["price_diff"] = asset.close.diff()
benchmark["price_diff"] = benchmark.close.diff()

Adds a new column 'gain' that indicates whether the price difference is positive

In [11]:
asset["gain"] = asset.price_diff > 0
benchmark["gain"] = benchmark.price_diff > 0

Adds a new column 'symbol' with the equity symbol

In [12]:
asset["symbol"] = "AAPL"
benchmark["symbol"] = "SPY"

Sets the volume of the 10th row to the mean volume of rows 5 to 9 in 'asset_2'

In [13]:
asset_2 = asset.copy()
asset_2.at[asset_2.index[10], "volume"] = asset_2.volume[asset_2.index[5:10]].mean()

In [14]:
asset_2.iat[10, 5]

0.0

Concatenates 'asset' and 'asset_2' and drops duplicates

In [15]:
df = pd.concat([asset, asset_2]).drop_duplicates()

Adds a new column 'returns' that contains the percentage change in closing prices

In [16]:
asset["returns"] = asset["close"].pct_change()

Creates a pivot table from 'asset' with 'returns' as values, 'gain' as columns, and 'sum', 'mean', 'std' as aggregation functions

In [17]:
df = pd.pivot_table(
    data=asset, values="returns", columns="gain", aggfunc=["sum", "mean", "std"]
)

In [18]:
display(df)

sum               mean                 std          
gain        False    True      False     True      False     True 
returns -1.173091  1.34348 -0.009858  0.010178  0.008515  0.010122

Concatenates 'asset' and 'benchmark' and groups by 'symbol' to calculate open, high, low, close (OHLC) prices

In [19]:
concated = pd.concat([asset, benchmark])

In [20]:
df = concated.groupby("symbol").close.ohlc()

In [21]:
display(df)

,open,high,low,close
symbol,,,,
AAPL,185.009995,216.669998,165.000000,214.289993
SPY,437.179993,548.489990,410.679993,548.489990


Fetches options chains for the symbol "AAPL" using the "cboe" provider and stores it in 'chains'

In [22]:
chains = obb.derivatives.options.chains("AAPL", provider="cboe")

In [23]:
display(chains)

,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
0,AAPL240621C00005000,2024-06-21,5.0,call,70,0,209.2267,186.82,up,208.30,...,0.000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0006,2024-05-30 15:17:02,2
1,AAPL240621P00005000,2024-06-21,5.0,put,35,6,0.0000,0.01,no_change,0.00,...,0.005,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2024-06-18 13:34:12,2
2,AAPL240621C00010000,2024-06-21,10.0,call,0,0,204.2299,171.61,down,203.40,...,0.000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0010,2024-02-26 15:49:52,2
3,AAPL240621P00010000,2024-06-21,10.0,put,1,0,0.0000,0.01,up,0.00,...,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2024-06-14 10:14:16,2
4,AAPL240621C00015000,2024-06-21,15.0,call,0,0,199.2330,156.73,down,198.50,...,0.000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0017,2024-03-28 09:31:59,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,AAPL261218P00330000,2026-12-18,330.0,put,0,0,115.7950,134.83,no_change,113.50,...,0.000,0.0000,0.2884,-1.0000,0.0000,-0.0285,0.0000,0.0000,2024-06-06 15:39:43,912
2122,AAPL261218C00340000,2026-12-18,340.0,call,174,21,8.2353,8.20,up,6.95,...,-0.350,-0.0409,0.2340,0.2210,0.0038,-0.0113,1.0073,0.9640,2024-06-18 14:41:03,912
2123,AAPL261218P00340000,2026-12-18,340.0,put,0,14,125.7950,126.45,up,123.55,...,3.775,0.0308,0.3039,-1.0000,0.0000,-0.0285,0.0000,0.0000,2024-06-18 11:18:51,912
2124,AAPL261218C00350000,2026-12-18,350.0,call,2604,128,7.2132,7.40,no_change,7.10,...,-0.025,-0.0034,0.2348,0.1989,0.0036,-0.0107,0.9493,0.8726,2024-06-18 15:33:39,912


Extracts unique expiration dates from 'chains'

In [24]:
expirations = chains.expiration.unique()

Filters 'chains' to get call options expiring on the 6th expiration date and stores it in 'calls'

In [25]:
calls = chains[(chains.option_type == "call") & (chains.expiration == expirations[5])]

Filters 'chains' to get put options expiring on the 6th expiration date and stores it in 'puts'

In [26]:
puts = chains[(chains.option_type == "put") & (chains.expiration == expirations[5])]

Sets the index of 'calls' and 'puts' to 'strike'

In [27]:
calls_strike = calls.set_index("strike")
puts_strike = puts.set_index("strike")

Joins 'calls_strike' and 'puts_strike' on the 'strike' index with a left join, adding suffixes to overlapping column names

In [28]:
joined = calls_strike.join(puts_strike, how="left", lsuffix="_call", rsuffix="_put")

Extracts the last trade prices of call and put options and stores them in 'prices'

In [29]:
prices = joined[["last_trade_price_call", "last_trade_price_put"]]

In [30]:
display(prices)

,last_trade_price_call,last_trade_price_put
strike,,
100.0,118.76,0.06
105.0,0.00,0.00
110.0,0.00,0.06
115.0,0.00,0.00
120.0,0.00,0.00
125.0,0.00,0.00
130.0,82.34,0.00
135.0,0.00,0.03
140.0,78.83,0.21


Adds a new column 'straddle_price' that contains the sum of last trade prices of call and put options

In [31]:
prices.loc[:, "straddle_price"] = prices.sum(axis=1)

In [32]:
display(prices)

,last_trade_price_call,last_trade_price_put,straddle_price
strike,,,
100.0,118.76,0.06,118.82
105.0,0.00,0.00,0.00
110.0,0.00,0.06,0.06
115.0,0.00,0.00,0.00
120.0,0.00,0.00,0.00
125.0,0.00,0.00,0.00
130.0,82.34,0.00,82.34
135.0,0.00,0.03,0.03
140.0,78.83,0.21,79.04


Groups 'chains' by option type, strike, and expiration to calculate the sum of open interest

In [33]:
df = chains.groupby(["option_type", "strike", "expiration"]).open_interest.sum()

In [34]:
display(df)

option_type  strike  expiration
call         5.0     2024-06-21    70
                     2024-07-19    14
                     2024-08-16     1
                     2024-09-20     4
                     2024-10-18     0
                                   ..
put          370.0   2024-08-16     0
                     2024-10-18     0
                     2024-11-15     0
                     2025-03-21     0
             380.0   2025-03-21     0
Name: open_interest, Length: 2126, dtype: int64

Groups 'chains' by option type, strike, and expiration to calculate the max last trade price and the sum of open interest

In [35]:
df = chains.groupby(["option_type", "strike", "expiration"]).agg(
    {"last_trade_price": "max", "open_interest": "sum"}
)

In [36]:
display(df)

last_trade_price  open_interest
option_type strike expiration                                 
call        5.0    2024-06-21            186.82             70
                   2024-07-19            210.74             14
                   2024-08-16            182.23              1
                   2024-09-20            208.94              4
                   2024-10-18            172.06              0
...                                         ...            ...
put         370.0  2024-08-16              0.00              0
                   2024-10-18              0.00              0
                   2024-11-15            187.00              0
                   2025-03-21              0.00              0
            380.0  2025-03-21            189.89              0

[2126 rows x 2 columns]

Groups 'chains' by option type and calculates the mean bid-ask spread for each group

In [38]:
df = chains.groupby(["option_type"]).apply(
    lambda x: (x["ask"] - x["bid"]).mean()
)

In [39]:
display(df)

option_type
call    1.818636
put     1.132643
dtype: float64

Standardizes the last trade prices by subtracting the mean and dividing by the standard deviation within each expiration group

In [40]:
(
    chains.groupby("expiration").last_trade_price.transform(
        lambda x: (x - x.mean()) / x.std()
    )
)

0       2.965133
1      -0.662307
2       2.669788
3      -0.662307
4       2.380851
          ...   
2121    1.734031
2122   -0.840912
2123    1.563629
2124   -0.857180
2125    1.725288
Name: last_trade_price, Length: 2126, dtype: float64

In [41]:
display(chains)

,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
0,AAPL240621C00005000,2024-06-21,5.0,call,70,0,209.2267,186.82,up,208.30,...,0.000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0006,2024-05-30 15:17:02,2
1,AAPL240621P00005000,2024-06-21,5.0,put,35,6,0.0000,0.01,no_change,0.00,...,0.005,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2024-06-18 13:34:12,2
2,AAPL240621C00010000,2024-06-21,10.0,call,0,0,204.2299,171.61,down,203.40,...,0.000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0010,2024-02-26 15:49:52,2
3,AAPL240621P00010000,2024-06-21,10.0,put,1,0,0.0000,0.01,up,0.00,...,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2024-06-14 10:14:16,2
4,AAPL240621C00015000,2024-06-21,15.0,call,0,0,199.2330,156.73,down,198.50,...,0.000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0017,2024-03-28 09:31:59,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,AAPL261218P00330000,2026-12-18,330.0,put,0,0,115.7950,134.83,no_change,113.50,...,0.000,0.0000,0.2884,-1.0000,0.0000,-0.0285,0.0000,0.0000,2024-06-06 15:39:43,912
2122,AAPL261218C00340000,2026-12-18,340.0,call,174,21,8.2353,8.20,up,6.95,...,-0.350,-0.0409,0.2340,0.2210,0.0038,-0.0113,1.0073,0.9640,2024-06-18 14:41:03,912
2123,AAPL261218P00340000,2026-12-18,340.0,put,0,14,125.7950,126.45,up,123.55,...,3.775,0.0308,0.3039,-1.0000,0.0000,-0.0285,0.0000,0.0000,2024-06-18 11:18:51,912
2124,AAPL261218C00350000,2026-12-18,350.0,call,2604,128,7.2132,7.40,no_change,7.10,...,-0.025,-0.0034,0.2348,0.1989,0.0036,-0.0107,0.9493,0.8726,2024-06-18 15:33:39,912


**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.